In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

class ModerateCNN(nn.Module):
    def __init__(self, dropout_rate=0.5):
        super(ModerateCNN, self).__init__()
        
        self.features = nn.Sequential(
            # First block
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.Dropout2d(0.2),
            
            # Second block
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.Dropout2d(0.3),
            
            # Third block
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.Dropout2d(0.4),
        )
        
        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d((7, 7)),
            nn.Flatten(),
            nn.Linear(128 * 7 * 7, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate),
            nn.Linear(512, 2)
        )
    
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

In [2]:
from torch.utils.data import Dataset
import os
from PIL import Image

class CustomDataset(Dataset):
    def __init__(self, data_folder, transform=None):
        self.data_folder = data_folder
        self.transform = transform

        self.class_names = sorted(os.listdir(data_folder))
        self.class_to_idx = {class_name: idx for idx, class_name in enumerate(self.class_names)}
        self.image_paths = []
        self.labels = []
        self.data = []

        for class_name in self.class_names:
            class_folder = os.path.join(data_folder, class_name)
            class_label = self.class_to_idx[class_name]
            for filename in os.listdir(class_folder):
                img_path = os.path.join(class_folder, filename)
                self.image_paths.append(img_path)
                self.labels.append(class_label)
                image = Image.open(img_path)
                self.data.append(np.array(image)) 

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path)
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

In [3]:
from torchvision import transforms

transform_valid = transforms.Compose([
            transforms.ToTensor(),
            transforms.ToPILImage(),          
            transforms.Resize((224, 224)),    
            transforms.Grayscale(num_output_channels=1),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.485],
                std=[0.229]
            ),
        ])

In [4]:
model = ModerateCNN()

model_data = './models/best_FL_global_model.pt' # path del best_local_model.pt

state_dict = torch.load(model_data)
model.load_state_dict(state_dict['model'])
model.eval()

/tmp/ipykernel_171059/403508705.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(model_data)


ModerateCNN(
  (features): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Dropout2d(p=0.2, inplace=False)
    (5): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): ReLU(inplace=True)
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Dropout2d(p=0.3, inplace=False)
    (10): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Dropout2d(p=0.4, inplace=False)
  )


In [ ]:
from torch.utils.data import DataLoader
import os

test_data_folder = "./test"
test_dataset = CustomDataset(test_data_folder, transform=transform_valid)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)

model.eval()
correct_predictions = 0
total_predictions = 0

with torch.no_grad():
    for i, (inputs, labels) in enumerate(test_loader):
        if i >= 1000:  # Break after processing 1000 images
            break
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)

        predicted_label = predicted.item()
        predicted_class_name = test_dataset.class_names[predicted_label]
        true_label = labels.item()
        true_class_name = test_dataset.class_names[true_label]

        print(f"predicted: {predicted_label} - {predicted_class_name}, actual: {true_label} - {true_class_name}")

        total_predictions += 1
        if predicted_label == true_label:
            correct_predictions += 1

accuracy = correct_predictions / total_predictions
print(f"\nAccuracy: {accuracy*100:.2f}%")

predicted: 1 - PNEUMONIA, actual: 1 - PNEUMONIA
predicted: 1 - PNEUMONIA, actual: 1 - PNEUMONIA
predicted: 1 - PNEUMONIA, actual: 0 - NORMAL
predicted: 1 - PNEUMONIA, actual: 0 - NORMAL
predicted: 1 - PNEUMONIA, actual: 1 - PNEUMONIA
predicted: 1 - PNEUMONIA, actual: 1 - PNEUMONIA
predicted: 1 - PNEUMONIA, actual: 0 - NORMAL
predicted: 1 - PNEUMONIA, actual: 1 - PNEUMONIA
predicted: 1 - PNEUMONIA, actual: 1 - PNEUMONIA
predicted: 1 - PNEUMONIA, actual: 0 - NORMAL
predicted: 1 - PNEUMONIA, actual: 1 - PNEUMONIA
predicted: 1 - PNEUMONIA, actual: 1 - PNEUMONIA
predicted: 1 - PNEUMONIA, actual: 1 - PNEUMONIA
predicted: 1 - PNEUMONIA, actual: 1 - PNEUMONIA
predicted: 1 - PNEUMONIA, actual: 1 - PNEUMONIA
predicted: 1 - PNEUMONIA, actual: 1 - PNEUMONIA
predicted: 1 - PNEUMONIA, actual: 1 - PNEUMONIA
predicted: 1 - PNEUMONIA, actual: 1 - PNEUMONIA
predicted: 0 - NORMAL, actual: 0 - NORMAL
predicted: 1 - PNEUMONIA, actual: 1 - PNEUMONIA
predicted: 1 - PNEUMONIA, actual: 1 - PNEUMONIA
predicted: